In [0]:
from pyspark.sql.functions import array_contains, size

class OnlyAirPodsAndIphoneTransformer(Transformer):
    """
    Customer who have bought only iphone or only airpods 
    """
    def transform(self, MyDF):
        transactionInputDF = MyDF.get("transactionInputDf")
        
        print("Transaction Input DF")
        transactionInputDF.show()

        groupedDF = transactionInputDF.groupBy("customer_id").agg(
            collect_set("product_name").alias("products")
        )

        groupedDF.show()
    
        # Assuming the requirement is to find customers who bought either only iPhones or only AirPods
        filteredDF = groupedDF.filter(
            (array_contains(col("products"), "iPhone")) &
            (array_contains(col("products"), "AirPods")) &
            (size(col("products")) == 2)
            
            )
             
            
        
        
        print("Only AirPods or iPhone")
        filteredDF.show()

        customerInputDF = MyDF.get("customerInputDF")
        customerInputDF.show()

        print("Joined DF")
        joinDF = customerInputDF.join(
            broadcast(filteredDF), "customer_id"
        )

        joinDF.select("customer_id", "customer_name", "location").show()

        return joinDF
